# Week 10 — Forecasting & Uncertainty (간단 예측/예측구간)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 다음 달 매출을 간단한 기준선으로 예측하고 불확실성을 함께 제시할 수 있는가?
- 예측값을 '약속'이 아니라 '범위'로 커뮤니케이션할 수 있는가?

## 2) Why this matters (Theory)
- 현업에서는 복잡한 모델보다 baseline(이동평균/계절성)부터
- 예측은 point estimate + interval로 전달
- 불확실성은 의사결정의 리스크 관리 도구

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 Baseline forecast: last-28-day mean as next-7-day forecast
✅ 단순하지만 강력한 baseline을 먼저 만듭니다.

In [ ]:
daily = (df.assign(revenue=df['revenue'].fillna(0))
           .groupby('date')['revenue'].sum()
           .sort_index())

last28 = daily.iloc[-28:]
forecast_mean = last28.mean()
print("baseline forecast (next day):", forecast_mean)

# visualize
plt.figure()
plt.plot(daily.index, daily.values, linewidth=0.7)
plt.axhline(forecast_mean, linewidth=2.0)
plt.title("Baseline forecast = last 28-day mean")
plt.show()

In [ ]:
one_sentence_insight('baseline 예측을 사용하면 어떤 의사결정을 더 빠르게 할 수 있는지 한 문장')

### 5.2 Forecast interval via bootstrap on last-28-day distribution
✅ 간단 예측구간: 마지막 28일 분포에서 부트스트랩으로 평균 범위 추정

In [ ]:
r = last28.values
rng = np.random.default_rng(0)
means = []
for _ in range(5000):
    rs = rng.choice(r, size=len(r), replace=True)
    means.append(rs.mean())
means = np.array(means)
lo, hi = np.quantile(means, [0.1, 0.9])  # 80% interval as practical band
print("80% forecast band for daily mean:", (lo, hi))

In [ ]:
one_sentence_insight("예측구간을 같이 제시하면 '과잉 반응'을 줄이는 이유를 한 문장")

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
